In [1]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# Load data
df = pd.read_csv("telecom_churn.csv")
df['date_of_registration'] = pd.to_datetime(df['date_of_registration'])
df['tenure_days'] = (pd.Timestamp.today() - df['date_of_registration']).dt.days
df['tenure_months'] = df['tenure_days'] // 30
df['age_group'] = pd.cut(df['age'], bins=[0, 25, 35, 45, 60, 100],
                         labels=["<25", "25-34", "35-44", "45-59", "60+"])

# Precomputed figures
fig1 = px.histogram(df, x='telecom_partner', color='gender', barmode='group',
                    title='Gender Distribution Across Telecom Partners')

state_usage = df.groupby('state')['data_used'].mean().reset_index()
fig2 = px.bar(state_usage, x='state', y='data_used',
              title='Average Data Usage by State')

fig3 = px.box(df, x='churn', y='estimated_salary', color='churn',
              title='Estimated Salary by Churn Status')

usage_cols = ['calls_made', 'sms_sent', 'data_used']
avg_usage = df.groupby('age_group')[usage_cols].mean().reset_index()
fig4 = px.line(avg_usage, x='age_group', y=usage_cols, markers=True,
               title='Average Usage Behavior by Age Group')

churn_rate = df.groupby('telecom_partner')['churn'].mean().reset_index()
fig5 = px.bar(churn_rate, x='telecom_partner', y='churn',
              title='Churn Rate by Telecom Partner')

tenure_churn = df.groupby('tenure_months')['churn'].mean().reset_index()
fig6 = px.line(tenure_churn, x='tenure_months', y='churn',
               title='Churn Rate Over Customer Tenure')

# Dash App
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Telecom Customer Churn Analysis Dashboard"),
    dcc.Tabs([
        dcc.Tab(label='Gender Distribution', children=[dcc.Graph(figure=fig1)]),
        dcc.Tab(label='Data Usage by State', children=[dcc.Graph(figure=fig2)]),
        dcc.Tab(label='Salary vs Churn', children=[dcc.Graph(figure=fig3)]),
        dcc.Tab(label='Usage by Age Group', children=[dcc.Graph(figure=fig4)]),
        dcc.Tab(label='Churn by Partner', children=[dcc.Graph(figure=fig5)]),
        dcc.Tab(label='Churn Over Tenure', children=[dcc.Graph(figure=fig6)])
    ])
])

if __name__ == '__main__':
    app.run(debug=True)
